In [11]:
import sys
sys.path.append("../../src")
import os
import datetime
import pandas as pd
import numpy as np

import scipy.io as sio
#from sindy_utils import library_size
from scipy.special import binom
from scipy.integrate import odeint

import tensorflow as tf

In [12]:
def get_rd_data(random=True):
    data = sio.loadmat('reaction_diffusion.mat')

    n_samples = data['t'].size
    n = data['x'].size
    N = n*n

    data['uf'] += 1e-6*np.random.randn(data['uf'].shape[0], data['uf'].shape[1], data['uf'].shape[2])
    data['duf'] += 1e-6*np.random.randn(data['duf'].shape[0], data['duf'].shape[1], data['duf'].shape[2])

    if not random:
        # consecutive samples
        training_samples = np.arange(int(.8*n_samples))
        val_samples = np.arange(int(.8*n_samples), int(.9*n_samples))
        test_samples = np.arange(int(.9*n_samples), n_samples)
    else:
        # random samples
        perm = np.random.permutation(int(.9*n_samples))
        training_samples = perm[:int(.8*n_samples)]
        val_samples = perm[int(.8*n_samples):]

        test_samples = np.arange(int(.9*n_samples), n_samples)

    training_data = {'t': data['t'][training_samples],
                     'y1': data['x'].T,
                     'y2': data['y'].T,
                     'x': data['uf'][:,:,training_samples].reshape((N,-1)).T,
                     'dx': data['duf'][:,:,training_samples].reshape((N,-1)).T}
    val_data = {'t': data['t'][val_samples],
                'y1': data['x'].T,
                'y2': data['y'].T,
                'x': data['uf'][:,:,val_samples].reshape((N,-1)).T,
                'dx': data['duf'][:,:,val_samples].reshape((N,-1)).T}
    test_data = {'t': data['t'][test_samples],
                 'y1': data['x'].T,
                 'y2': data['y'].T,
                 'x': data['uf'][:,:,test_samples].reshape((N,-1)).T,
                 'dx': data['duf'][:,:,test_samples].reshape((N,-1)).T}

    return training_data, val_data, test_data

In [13]:
def library_size(n, poly_order, use_sine=False, include_constant=True):
    l = 0
    for k in range(poly_order+1):
        l += int(binom(n+k-1,k))
    if use_sine:
        l += n
    if not include_constant:
        l -= 1
    return l

# Generate data

In [14]:
import pickle

training_data, validation_data, test_data = get_rd_data() 

pickle.dump(training_data, open('training_data.pkl', 'wb'))
pickle.dump(validation_data, open('validation_data.pkl', 'wb'))
pickle.dump(test_data, open('test_data.pkl', 'wb'))

# Set up model and training parameters

In [27]:
params = {}

params['input_dim'] = training_data['y1'].size*training_data['y2'].size
params['latent_dim'] = 2
params['model_order'] = 1
params['poly_order'] = 3
params['include_sine'] = True
params['library_dim'] = library_size(params['latent_dim'], params['poly_order'], params['include_sine'], True)

# sequential thresholding parameters
params['sequential_thresholding'] = True
params['coefficient_threshold'] = 0.1
params['threshold_frequency'] = 500
params['coefficient_mask'] = np.ones((params['library_dim'], params['latent_dim']))
params['coefficient_initialization'] = 'constant'

# loss function weighting
params['loss_weight_decoder'] = 1.0
params['loss_weight_sindy_z'] = 0.01
params['loss_weight_sindy_x'] = 0.5
params['loss_weight_sindy_regularization'] = 0.1

params['activation'] = 'sigmoid'
params['widths'] = [256]

# training parameters
params['epoch_size'] = training_data['t'].size
params['batch_size'] = 1000
params['learning_rate'] = 1e-3

params['data_path'] = os.getcwd() + '/'
params['print_progress'] = True
params['print_frequency'] = 100

# training time cutoffs
params['max_epochs'] = 3001
params['refinement_epochs'] = 1001

# Run training experiments

In [20]:
num_experiments = 10
df = pd.DataFrame()
for i in range(num_experiments):
    print('EXPERIMENT %d' % i)

    params['coefficient_mask'] = np.ones((params['library_dim'], params['latent_dim']))

    params['save_name'] = 'rd_' + datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S_%f")

    tf.reset_default_graph()

    results_dict = train_network(training_data, validation_data, params)
    df = df.append({**results_dict, **params}, ignore_index=True)

df.to_pickle('experiment_results_' + datetime.datetime.now().strftime("%Y%m%d%H%M") + '.pkl')

EXPERIMENT 0


AttributeError: module 'tensorflow' has no attribute 'reset_default_graph'